In [1]:
import numpy as np
import pandas as pd

# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt
# %matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

In [2]:
with open('../../results/full_search/load=3482_pv=7989 6423/12_26_16_30_daysinchunk=365_conf=0.95_epsilon=0.19.csv') as file:
    full_df = pd.read_csv(file, sep=',')
    full_df['color_cost'] = np.log(full_df['cost'] - min(full_df['cost']) + 1)
    full_df['chunk_start_round'] = (full_df['chunk_start'] % 8760) // 24

In [5]:
px.scatter_3d(full_df, x='pv1', y='pv2', z='battery', color='type', text='chunk_start_round')

In [6]:
px.scatter_3d(full_df[full_df['type'] == 0], x='pv1', y='pv2', z='battery',
              color='cost', text='chunk_start_round')

In [93]:
def calc_cheby_2(cheby_df, epsilon=0.05, cols=['pv1', 'pv2', 'battery'], pv_max=[50, 50], cell_max=50):
    
    def calc_lambda(zeta, mu_eta):
        return (zeta - mu_eta) @ sigma_eta_inv @ (zeta - mu_eta)
    
    def calc_cost(pv1, pv2, cell):
        pv1_cost = pv1 * 500 + 2000 if pv1 > 0 else 0
        pv2_cost = pv2 * 500 + 2000 if pv2 > 0 else 0
        cell_cost = cell * 500
        return pv1_cost + pv2_cost + cell_cost
    
    N = 2
    assert len(cols) == N + 1
    assert len(pv_max) == N
    
    eta = len(cheby_df)

    sigma = cheby_df[cols].to_numpy()
    mu_eta = np.mean(sigma, axis=0)
#     print(mu_eta)

    sigma_eta = np.dot(np.transpose(sigma - mu_eta), sigma - mu_eta) / (eta - 1)
#     print(sigma_eta)

    sigma_eta_inv = np.linalg.inv(sigma_eta)
#     print(sigma_eta_inv)

    lambda2 = (eta**2 - 1) / (epsilon * eta**2 / (N + 1) - eta)
#     print(lambda2)

    cheby_pts = []
    min_cost_pair = (-1, -1, -1, float('inf'))
    
    for pv1 in np.arange(mu_eta[0], pv_max[0], pv_max[0] / 400):
        for pv2 in np.arange(mu_eta[1], pv_max[1], pv_max[1] / 400):
            cell_lo = mu_eta[-1]
            cell_hi = cell_max
            while cell_hi - cell_lo > (cell_max / 400):
                cell_mid = (cell_lo + cell_hi) / 2
                if calc_lambda([pv1, pv2, cell_mid], mu_eta) > lambda2:
                    cell_hi = cell_mid
                else:
                    cell_lo = cell_mid
            if (cell_hi > mu_eta[-1] + (cell_max / 400)):
                xi = (pv1, pv2, cell_hi)
                cost = calc_cost(*xi)
                xi_and_cost = (*xi, cost)
                if cost < min_cost_pair[-1]:
                    min_cost_pair = xi_and_cost
                cheby_pts.append(xi_and_cost)
            
    cheby_pts_df = pd.DataFrame(cheby_pts)
    fig = px.scatter_3d(cheby_df, x='pv1', y='pv2', z='battery', color='cost')
    fig.add_trace(px.scatter_3d(cheby_pts_df, x=0, y=1, z=2, color=3).data[0])
    fig.show()

    max_cost_pair = cheby_pts_df[cheby_pts_df[3] == max(cheby_pts_df[3])].iloc[0]
    
    return max_cost_pair

In [94]:
calc_cheby_2(full_df[full_df['type'] == 0], epsilon=0.19, pv_max=[20, 20])

0        5.781244
1       11.178267
2       41.502959
3    33231.235029
Name: 56, dtype: float64

In [87]:
def calc_cheby_1(cheby_df, epsilon=0.05, cols=['pv1', 'battery'], pv_max=[50], cell_max=50):
    
    def calc_lambda(zeta):
        return (zeta - mu_eta) @ sigma_eta_inv @ (zeta - mu_eta)
    
    def calc_cost(pv1, cell):
        pv1_cost = pv1 * 500 + 2000 if pv1 > 0 else 0
        cell_cost = cell * 500
        return pv1_cost + cell_cost
    
    N = 1
    assert len(cols) == N + 1
    assert len(pv_max) == N
    
    eta = len(cheby_df)

    sigma = cheby_df[cols].to_numpy()
    mu_eta = np.mean(sigma, axis=0)
#     print(mu_eta)

    sigma_eta = np.dot(np.transpose(sigma - mu_eta), sigma - mu_eta) / (eta - 1)
#     print(sigma_eta)

    sigma_eta_inv = np.linalg.inv(sigma_eta)
#     print(sigma_eta_inv)

    lambda2 = (eta**2 - 1) / (epsilon * eta**2 / (N + 1) - eta)
#     print(lambda2)

    cheby_pts = []
    min_cost_pair = (-1, -1, float('inf'))
    
    for pv1 in np.arange(mu_eta[0], pv_max[0], pv_max[0] / 400):
        cell_lo = mu_eta[-1]
        cell_hi = cell_max
        while cell_hi - cell_lo > (cell_max / 400):
            cell_mid = (cell_lo + cell_hi) / 2
            if calc_lambda([pv1, cell_mid]) > lambda2:
                cell_hi = cell_mid
            else:
                cell_lo = cell_mid
        if (cell_hi > mu_eta[-1] + (cell_max / 400)):
            xi = (pv1, cell_hi)
            cost = calc_cost(*xi)
            xi_and_cost = (*xi, cost)
            if cost < min_cost_pair[-1]:
                min_cost_pair = xi_and_cost
            cheby_pts.append(xi_and_cost)
            
    cheby_pts_df = pd.DataFrame(cheby_pts)
    fig = px.scatter(cheby_df, x=cols[0], y=cols[-1], color='cost')
    fig.add_trace(px.scatter(cheby_pts_df, x=0, y=1, color=2).data[0])
    fig.show()
    
    max_cost_pair = cheby_pts_df[cheby_pts_df[2] == max(cheby_pts_df[2])].iloc[0]
    
    return max_cost_pair

In [88]:
calc_cheby_1(full_df[full_df['type'] == 1], epsilon=0.19, pv_max=[50])

0       17.791864
1       41.234159
2    31513.011225
Name: 21, dtype: float64

In [89]:
calc_cheby_1(full_df[full_df['type'] == 2], cols=['pv2', 'battery'], epsilon=0.19, pv_max=[50])

0       17.926667
1       39.213264
2    30569.965308
Name: 25, dtype: float64